In [1]:
from os import environ
environ['CUDA_VISIBLE_DEVICES'] = '3'

In [30]:
from __future__ import print_function
import json
from pprint import pprint
import numpy as np
from functools import reduce
import re
from nltk.tokenize import word_tokenize
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, Bidirectional, Dense, Flatten, Dropout, LSTM, GRU, Conv1D, MaxPooling1D, UpSampling1D,BatchNormalization
from keras.layers import Activation
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
import string
from keras.models import load_model

import keras

In [3]:
def normalize(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_dictionary(vocab):
    d = dict()
    with open(vocab, 'r') as f:
        lines = f.readlines()
        for l in lines:
            w = l.strip().split()
            k = w[0]
            v = []
            for i in range(1, len(w)):
                v.append(float(w[i]))
            d[k] = np.array(v, dtype=np.float32)
    return d
def tokenize_sequence(seq, max_length, dim):
    words = word_tokenize(seq.lower())
    word_vectors = list(map(lambda x: d[x] if x in d else np.zeros(dim), words))
    for i in range(len(word_vectors), max_length):
        word_vectors.append(np.zeros(dim))
    word_vectors = word_vectors[:max_length]
    return np.array(word_vectors)
def create_one_hot_answer(para, answer, answer_start, option, max_length):
    if option == "s":
        from_begin = para[0:answer_start]
    else:
        from_begin = para[0:answer_start+len(answer)]
    l = len(word_tokenize(from_begin))
    one_hot = np.zeros(max_length)
    if option == "s":
        one_hot[min(l, max_length - 1)] = 1
    else:
        one_hot[min(l - 1, max_length-1)] = 1
    return one_hot

In [4]:
d = get_dictionary("glove/glove.6B.200d.txt")

In [5]:
with open('train.json') as data_file:
    data = json.load(data_file)
ids = []
titles = dict()
contexts = dict()
questions = dict()
answers_text = dict()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
            id_ = qas[k]["id"]
            answer = qas[k]["answer"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts[id_] = context
            answers_start[id_] = answer["answer_start"]
            answers_text[id_] = answer["text"]
            questions[id_] = question

In [6]:
max_para = 664
max_q = 50
dimension = 200


In [7]:
t = [i for i in range(len(ids))]
t.sort(key=lambda x: np.random.random())
ind = t[:len(questions)]

In [8]:
paras = [tokenize_sequence(contexts[ids[ind[i]]],max_para,dimension) for i in range(len(ind))]
qns = [tokenize_sequence(questions[ids[ind[i]]],max_q,dimension) for i in range(len(ind))]
ans_starts = [create_one_hot_answer(contexts[ids[ind[i]]], answers_text[ids[ind[i]]], answers_start[ids[ind[i]]], "s", max_para) for i in range(len(ind))]
ans_ends = [create_one_hot_answer(contexts[ids[ind[i]]], answers_text[ids[ind[i]]], answers_start[ids[ind[i]]], "e", max_para) for i in range(len(ind))]


In [9]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 50
BATCH_SIZE = 512
EPOCHS = 10

In [9]:
sentence = layers.Input(shape=(max_para,dimension), dtype='float32')
encoded_sentence = GRU(200, return_sequences=True)(sentence)
print(encoded_sentence.shape)
question = layers.Input(shape=(max_q,dimension), dtype='float32')
encoded_question = GRU(200, return_sequences=True)(question)

(?, ?, 200)


In [10]:
merge_1 = layers.dot([encoded_sentence, encoded_question], axes = 2 )
A_Q = layers.Activation("softmax")(merge_1)
merge_2 = layers.dot([encoded_question, encoded_sentence], axes = 2 )
A_D = layers.Activation("softmax")(merge_2)
C_Q = layers.dot([A_Q, encoded_sentence], axes = 1 )
print(C_Q.shape)

(?, ?, 200)


In [11]:
C_Q = layers.concatenate([encoded_question, C_Q], axis=2)
C_D = layers.dot([A_D, C_Q], axes=1)
C_ = layers.concatenate([encoded_sentence, C_D], axis=2)
print(C_.shape)


(?, ?, 600)


In [12]:
U = Bidirectional(LSTM(100, return_sequences=True))(C_)
U = Dropout(0.5)(U)
print(U.shape)

(?, ?, 200)


In [13]:
#Convoluion, batch norm, relu unit
def convBN_pool(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 1, padding = 'valid', strides = 2)(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros')(convlayer)
    activation = layers.PReLU()(BN)

    return activation

def convBN(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 3, padding = 'same')(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation

def RU(input_layer, conv_channels, d_rate):
    # input tensor for a 3-channel 256x256 image
    x = input_layer
    # 3x3 conv with 3 output channels (same as input channels)
    y = Conv1D(conv_channels, 3, padding='same', dilation_rate = d_rate)(x)
    y = Conv1D(conv_channels, 3, padding='same', dilation_rate = d_rate)(y)
    # this returns x + y.
    z = layers.add([x, y])
    return z

In [14]:
start = GRU(100, return_sequences=True)(U)
start = convBN(start, 100)
start = RU(start, 100, 1)
start = Dropout(0.5)(start)
start = convBN_pool(start, 64)
start = RU(start, 64, 1)
start = convBN_pool(start, 64)
start = RU(start, 64, 2)
start = convBN_pool(start, 128)
start = RU(start, 128, 1)
start =layers.PReLU()(start)
start = RU(start, 128, 2)
start = Dropout(0.5)(start)
start = convBN_pool(start, 256)
start = RU(start, 256, 2)
start = Dropout(0.5)(start)
start = convBN_pool(start, 128)
start = RU(start, 128, 1)
start = layers.PReLU()(start)
start = Dropout(0.5)(start)
start = convBN(start, 64)
start = RU(start, 64, 1)
start = layers.PReLU()(start)
start = Flatten()(start)
start = Dropout(0.5)(start)
start = Dense(max_para, activation='softmax', name='output_1')(start)


end = GRU(100, return_sequences=True)(U)
end = convBN(end, 100)
end = RU(end, 100, 1)
end = Dropout(0.5)(end)
end = convBN_pool(end, 64)
end = RU(end, 64, 1)
end = convBN_pool(end, 64)
end = RU(end, 64, 2)
end = convBN_pool(end, 128)
end = RU(end, 128, 1)
end = layers.PReLU()(end)
end = RU(end, 128, 2)
end = Dropout(0.5)(end)
end = convBN_pool(end, 256)
end = RU(end, 256, 2)
end = Dropout(0.5)(end)
end = convBN_pool(end, 128)
end = RU(end, 128, 1)
end = layers.PReLU()(end)
end = Dropout(0.5)(end)
end = convBN(end, 64)
end = RU(end, 64, 1)
end = layers.PReLU()(end)
end = Flatten()(end)
end = Dropout(0.5)(end)
end = Dense(max_para, activation='softmax', name='output_2')(end)


In [15]:
model = Model([sentence, question],[start, end])
model.compile(optimizer='adam',
              loss={'output_1': 'categorical_crossentropy', 'output_2': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [16]:
del d
x = np.array(paras)
xq = np.array(qns)
ans_s = np.array(ans_starts)
ans_e = np.array(ans_ends)

In [17]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 664, 200)      0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 50, 200)       0                                            
____________________________________________________________________________________________________
gru_1 (GRU)                      (None, 664, 200)      240600                                       
____________________________________________________________________________________________________
gru_2 (GRU)                      (None, 50, 200)       240600                                       
___________________________________________________________________________________________

____________________________________________________________________________________________________
p_re_lu_18 (PReLU)               (None, 21, 128)       2688                                         
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 21, 128)       0                                            
____________________________________________________________________________________________________
dropout_10 (Dropout)             (None, 21, 128)       0                                            
____________________________________________________________________________________________________
conv1d_21 (Conv1D)               (None, 21, 64)        24640                                        
____________________________________________________________________________________________________
conv1d_44 (Conv1D)               (None, 21, 64)        24640                               

In [23]:
print('Training')
val_split = 0.1
for i in range(5):
    model.fit([x, xq], [ans_s, ans_e],
              batch_size=200,
              epochs=5,
              validation_split=val_split)
    if i == 1:
        val_split = 0
    model.save("model_third.h5")

Training
Train on 55241 samples, validate on 6138 samples
Epoch 1/5
55241/55241 [==============================] - 2005s - loss: 6.6874 - output_1_loss: 3.2997 - output_2_loss: 3.3877 - output_1_acc: 0.1078 - output_2_acc: 0.0974 - val_loss: 7.3954 - val_output_1_loss: 3.6404 - val_output_2_loss: 3.7550 - val_output_1_acc: 0.1160 - val_output_2_acc: 0.0925
Epoch 2/5
55241/55241 [==============================] - 1902s - loss: 6.3017 - output_1_loss: 3.1163 - output_2_loss: 3.1854 - output_1_acc: 0.1297 - output_2_acc: 0.1263 - val_loss: 7.3255 - val_output_1_loss: 3.7545 - val_output_2_loss: 3.5709 - val_output_1_acc: 0.1173 - val_output_2_acc: 0.1323
Epoch 3/5
55241/55241 [==============================] - 1902s - loss: 5.9416 - output_1_loss: 2.9581 - output_2_loss: 2.9835 - output_1_acc: 0.1541 - output_2_acc: 0.1697 - val_loss: 6.9941 - val_output_1_loss: 3.5152 - val_output_2_loss: 3.4789 - val_output_1_acc: 0.1527 - val_output_2_acc: 0.1867
Epoch 4/5
55241/55241 [================

In [12]:
model.save("model_third.h5")

In [11]:
model = load_model("model_third.h5")

In [36]:
def convBN_init(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 5, padding = 'same')(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation
def convBN(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 3, padding = 'same')(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation
def convBN_dilate(input_layer, conv_channels):
    convlayer = Conv1D(conv_channels, 3, padding = 'same', dilation_rate=2)(input_layer)
    BN = BatchNormalization(axis=-1, momentum = 0.99, epsilon=0.001, center=True, scale = True, 
                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                            beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)(convlayer)
    activation = layers.PReLU()(BN)

    return activation
def RU(input_layer, conv_channels):
    # input tensor for a 3-channel 256x256 image
    x = input_layer
    # 3x3 conv with 3 output channels (same as input channels)
    y = Conv1D(conv_channels, 3, padding='same')(x)
    y = Conv1D(conv_channels, 3, padding='same')(y)
    # this returns x + y.
    z = layers.add([x, y])
    return z
#Full resolution residual unit
def FRRU(input_z, input_y, conv_channels):
    #Calculate pooling size
    zshape = input_z.shape.as_list()[1]
    yshape = input_y.shape.as_list()[1]
   
    ps = int(zshape / yshape)

    downsampling_z = MaxPooling1D(pool_size = ps, strides = None, padding ='valid')(input_z)
  
    concat_yz = keras.layers.concatenate([input_y, downsampling_z])
    y_stream = convBN(concat_yz, conv_channels)
    y_stream = convBN(y_stream, conv_channels)

    upsampling_y = UpSampling1D(size=ps)(y_stream)
    upsampling_y = Conv1D(input_z.shape.as_list()[2], 1, padding = 'same')(upsampling_y)
    z_stream = keras.layers.add([upsampling_y,input_z])
    return z_stream, y_stream
def FRRU_dilate(input_z, input_y, conv_channels):
    #Calculate pooling size
    zshape = input_z.shape.as_list()[1]
    yshape = input_y.shape.as_list()[1]
   
    ps = int(zshape / yshape)

    downsampling_z = MaxPooling1D(pool_size = ps, strides = None, padding ='valid')(input_z)
  
    concat_yz = keras.layers.concatenate([input_y, downsampling_z])
    y_stream = convBN_dilate(concat_yz, conv_channels)
    y_stream = convBN_dilate(y_stream, conv_channels)

    upsampling_y = UpSampling1D(size=ps)(y_stream)
    upsampling_y = Conv1D(input_z.shape.as_list()[2], 1, padding = 'same')(upsampling_y)
    z_stream = keras.layers.add([upsampling_y,input_z])
    return z_stream, y_stream

In [60]:
sentence = layers.Input(shape=(max_para,dimension), dtype='float32')
encoded_sentence = GRU(200, return_sequences=True)(sentence)
print(encoded_sentence.shape)
question = layers.Input(shape=(max_q,dimension), dtype='float32')
encoded_question = GRU(200, return_sequences=True)(question)

merge_1 = layers.dot([sentence, question], axes = 2 )
A_Q = layers.Activation("softmax")(merge_1)
merge_2 = layers.dot([question, sentence], axes = 2 )
A_D = layers.Activation("softmax")(merge_2)
C_Q = layers.dot([A_Q, sentence], axes = 1 )
print(C_Q.shape)

C_Q = layers.concatenate([question, C_Q], axis=2)
C_D = layers.dot([A_D, C_Q], axes=1)
C_ = layers.concatenate([sentence, C_D], axis=2)
print(C_.shape)

(?, ?, 200)
(?, 50, 200)
(?, 664, 600)


In [61]:
#stream = Dropout(0.2)(input_layer)
stream = convBN_init(C_, 200)
stream = RU(stream, 200)
stream = RU(stream, 200)
# stream = RU(stream, 48)

stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream)
stream_z = Conv1D(200, 1, padding='same')(stream)

stream_z, stream_y = FRRU(stream_z, stream_y, 250)
stream_z, stream_y = FRRU(stream_z, stream_y, 250)
# stream_z, stream_y = FRRU(stream_z, stream_y, 400)
print(stream_y.shape.as_list())
stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream_y)

stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_z, stream_y = FRRU_dilate(stream_z, stream_y, 300)
stream_z, stream_y = FRRU_dilate(stream_z, stream_y, 300)
stream_y = Dropout(0.5)(stream_y)
# stream_z, stream_y = FRRU(stream_z, stream_y, 450)

# stream_z, stream_y = FRRU(stream_z, stream_y, 192)
print(stream_y.shape.as_list())
stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream_y)

stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_z, stream_y = FRRU(stream_z, stream_y, 300)
stream_y = Dropout(0.5)(stream_y)
stream_z = Dropout(0.5)(stream_z)
# stream_z, stream_y = FRRU(stream_z, stream_y, 384)
print(stream_y.shape.as_list())
# stream_y = MaxPooling1D(pool_size = 2, strides = None, padding ='valid')(stream_y)
# stream_z, stream_y = FRRU(stream_z, stream_y, 384)
# # stream_z, stream_y = FRRU(stream_z, stream_y, 384)
# print(stream_y.shape.as_list())
# stream_y = UpSampling1D(size=2)(stream_y)
# stream_z, stream_y = FRRU(stream_z, stream_y, 192)
# stream_z, stream_y = FRRU(stream_z, stream_y, 192)
# print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 250)
stream_y = Dropout(0.5)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 250)

print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 200)
stream_y = Dropout(0.5)(stream_y)
stream_z, stream_y = FRRU(stream_z, stream_y, 200)
print(stream_y.shape.as_list())
stream_y = UpSampling1D(size=2)(stream_y)
print(stream_y.shape.as_list())
print(stream_z.shape.as_list())
stream = keras.layers.concatenate([stream_y, stream_z])
stream = Conv1D(100, 1, padding='same')(stream)
stream = Dropout(0.5)(stream)
print(stream.shape.as_list())
# stream = RU(stream, 48)
stream = RU(stream, 100)

stream = RU(stream, 100)

start = Conv1D(1, 1, padding='same')(stream)
start = Flatten()(start)
start = Activation('softmax', name = 'output_1')(start)
end = Conv1D(1, 1, padding='same')(stream)
end = Flatten()(end)
end = Activation('softmax', name = 'output_2')(end)

[None, 332, 400]
[None, 166, 450]
[None, 83, 600]
[None, 166, 450]
[None, 332, 300]
[None, 664, 300]
[None, 664, 400]
[None, 664, 100]


In [62]:
model = Model([sentence, question],[start, end])
model.compile(optimizer='adam',
              loss={'output_1': 'categorical_crossentropy', 'output_2': 'categorical_crossentropy'},
              metrics=['accuracy'])

In [63]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, 664, 200)      0                                            
____________________________________________________________________________________________________
input_12 (InputLayer)            (None, 50, 200)       0                                            
____________________________________________________________________________________________________
dot_21 (Dot)                     (None, 664, 50)       0                                            
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 664, 50)       0                                            
___________________________________________________________________________________________

In [ ]:
print('Training')
val_split = 0.1
for i in range(5):
    model.fit([x, xq], [ans_s, ans_e],
              batch_size=64,
              epochs=5,
              validation_split=val_split)
    if i == 1:
        val_split = 0
    model.save("model_fourth.h5")

Training
Train on 55241 samples, validate on 6138 samples
Epoch 1/5
55241/55241 [==============================] - 1408s - loss: 6.8325 - output_1_loss: 3.4744 - output_2_loss: 3.3581 - output_1_acc: 0.1953 - output_2_acc: 0.2010 - val_loss: 6.0789 - val_output_1_loss: 3.1157 - val_output_2_loss: 2.9633 - val_output_1_acc: 0.2626 - val_output_2_acc: 0.2755
Epoch 2/5
55241/55241 [==============================] - 1401s - loss: 5.9244 - output_1_loss: 3.0463 - output_2_loss: 2.8781 - output_1_acc: 0.2717 - output_2_acc: 0.2891 - val_loss: 6.4206 - val_output_1_loss: 3.2898 - val_output_2_loss: 3.1308 - val_output_1_acc: 0.2911 - val_output_2_acc: 0.3107
Epoch 3/5
55241/55241 [==============================] - 1493s - loss: 5.4176 - output_1_loss: 2.8030 - output_2_loss: 2.6146 - output_1_acc: 0.3185 - output_2_acc: 0.3460 - val_loss: 5.7279 - val_output_1_loss: 2.9228 - val_output_2_loss: 2.8051 - val_output_1_acc: 0.3170 - val_output_2_acc: 0.3341
Epoch 4/5
55241/55241 [================

In [49]:
ans_s.shape

(61379, 664)

In [58]:
print('Training')
val_split = 0.1
for i in range(4):
    model.fit([x, xq], [ans_s, ans_e],
              batch_size=180,
              epochs=5,
              validation_split=val_split)
    if i == 1:
        val_split = 0
    model.save("model_fifth.h5")

Training
Train on 55241 samples, validate on 6138 samples
Epoch 1/5
55241/55241 [==============================] - 871s - loss: 3.5110 - output_1_loss: 1.8561 - output_2_loss: 1.6549 - output_1_acc: 0.5098 - output_2_acc: 0.5556 - val_loss: 5.7004 - val_output_1_loss: 2.9326 - val_output_2_loss: 2.7678 - val_output_1_acc: 0.3180 - val_output_2_acc: 0.3384
Epoch 2/5
 1800/55241 [..............................] - ETA: 707s - loss: 2.8964 - output_1_loss: 1.5521 - output_2_loss: 1.3443 - output_1_acc: 0.5778 - output_2_acc: 0.6333

KeyboardInterrupt: 